In [1]:
from __future__ import print_function, division

import copy
import time

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import models

import settings as opt
import utils
from preprocess.dataset import ValenceArousalWithClassesDataset

In [ ]:
cudnn.benchmark = True

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

print(device)

In [ ]:
# Image preprocessing

pre_process = transforms.Compose([
    transforms.Resize(opt.image_size),
    transforms.CenterCrop(opt.image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset = ValenceArousalWithClassesDataset(csv_file=opt.annonations_file,
                                           root_dir=opt.all_images_path,
                                           transform=pre_process)

train_set_size = int(0.8 * len(dataset))
val_set_size = len(dataset) - train_set_size
train_set, val_set = torch.utils.data.random_split(dataset, [train_set_size, val_set_size])

n_classes = opt.n_classes
class_names = list(opt.labels_map.values())

batch_size = 128

# Create the dataloader
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=batch_size, shuffle=True)

# Plot some training images
real_images, real_labels = next(iter(val_dataloader))
real_labels = real_labels.numpy().flatten()
real_labels = [opt.labels_map[label_class] for label_class in real_labels]
utils.plot_images_with_labels(real_images, real_labels, predicted_labels=False, device=device)

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader = train_dataloader
            else:
                model.eval()  # Set model to evaluate mode
                dataloader = val_dataloader

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device).view(-1)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()
                epoch_loss = running_loss / train_set_size
                epoch_acc = running_corrects.double() / train_set_size
            else:
                epoch_loss = running_loss / val_set_size
                epoch_acc = running_corrects.double() / val_set_size

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [5]:
model_ft = models.efficientnet_b7(pretrained=True)
num_ftrs = model_ft.classifier[1].in_features
model_ft.classifier[1] = nn.Linear(num_ftrs, 13)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=opt.lr_D, betas=(opt.beta1, opt.beta2))

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

In [ ]:
# Plot some training images
real_images, real_labels = next(iter(val_dataloader))
real_labels = real_labels.numpy().flatten()
real_labels = [opt.labels_map[label_class] for label_class in real_labels]

model_ft.eval()

real_images = real_images.to(device)

outputs = model_ft(real_images)
_, preds = torch.max(outputs, 1)

predicted_labels = preds.to('cpu').numpy().flatten()
predicted_labels = [opt.labels_map[label_class] for label_class in predicted_labels]

utils.plot_images_with_labels(real_images, real_labels, predicted_labels, device)